In [1]:
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
import os
from models.data_loader import DataLoader
from models.retain_bidirectional import RETAIN

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
# hyperparameters
batch_size = 50
max_seq_length = 300
min_seq_length = 5
num_classes = 267+1
cuda_flag = True

In [4]:
# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='val', max_seq_length=max_seq_length, min_seq_length=min_seq_length)


In [5]:
import numpy as np
for num in np.arange(500,6500,500):
    ver = 'retain_bi_%d.pth'%num
    model = torch.load(os.path.join('/home/mjc/github/EHRVis/data/saved_weights',ver))
    model.eval()
    # test model
    cnt = 0
    total = 0
    correct = 0
    correct_5 = 0
    for i,file in enumerate(D.val_list):
    # for i,file in enumerate(['2014_42.pckl']):
    #     print('[%d/%d] opening from %s'%(i+1,len(D.test_list),file))
        file_num = int(file.split('_')[1].split('.')[0])
        D.batch_size = int(20000/file_num)
        D.load_batch_file(file)
        for i in range(D.batch_count):
            cnt+=1
            input_list, targets = D.get_batch()
            inputs = model.list_to_tensor(input_list)
            outputs = model(inputs)
            targets = torch.LongTensor(targets).view(len(inputs),-1)[:,-1] # to only use last of each sequence
            if cuda_flag:
                targets = targets.cuda()
            total+=len(inputs)
            targets = targets.tolist()
            for tup in zip(targets,outputs.topk(5)[1].data.tolist()):
                if tup[0]==tup[1][0]:
                    correct +=1
                if tup[0] in tup[1]:
                    correct_5 +=1
    print(ver)
    print("Top-1 accuracy: %d/%d = %1.3f"%(correct,total,correct*1.0/total))
    print("Top-5 accuracy: %d/%d = %1.3f"%(correct_5,total,correct_5*1.0/total))

/home/mjc/github/EHRVis/models/retain_bidirectional.py:35: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  outputs1 = self.RNN1(embedded) # [b x seq x 128*2]
/home/mjc/github/EHRVis/models/retain_bidirectional.py:43: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  outputs2 = self.RNN2(embedded) # [b x seq x 128]


retain_bi_500.pth
Top-1 accuracy: 10384/37411 = 0.278
Top-5 accuracy: 21579/37411 = 0.577
retain_bi_1000.pth
Top-1 accuracy: 11372/37411 = 0.304
Top-5 accuracy: 23362/37411 = 0.624
retain_bi_1500.pth
Top-1 accuracy: 10818/37411 = 0.289
Top-5 accuracy: 22879/37411 = 0.612
retain_bi_2000.pth
Top-1 accuracy: 11107/37411 = 0.297
Top-5 accuracy: 23157/37411 = 0.619
retain_bi_2500.pth
Top-1 accuracy: 11587/37411 = 0.310
Top-5 accuracy: 23325/37411 = 0.623
retain_bi_3000.pth
Top-1 accuracy: 11065/37411 = 0.296
Top-5 accuracy: 23218/37411 = 0.621
retain_bi_3500.pth
Top-1 accuracy: 11111/37411 = 0.297
Top-5 accuracy: 23006/37411 = 0.615
retain_bi_4000.pth
Top-1 accuracy: 11398/37411 = 0.305
Top-5 accuracy: 23107/37411 = 0.618
retain_bi_4500.pth
Top-1 accuracy: 10441/37411 = 0.279
Top-5 accuracy: 22339/37411 = 0.597


KeyboardInterrupt: 